# TP noté 2 - *correction*

On utilise dans ce *notebook* les modules `pymongo`, `pandas` et `pprint`.

In [1]:
import pymongo
import pandas
import pprint

On créé ensuite la connexion vers *MongoDB*. Et on vérifie que la base `test` contient bien la collection `restaurants`.

In [2]:
con = pymongo.MongoClient("mongodb://193.51.82.104:2343/")
con.test.collection_names()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  


['restaurants', 'test', 'grades']

On va aussi utiliser ici la fonction `affiche()` créée dans le TP pour améliorer la lisibilité de l'affichage des résultats.

In [3]:
def affiche(res):
    for r in res:
        pprint.pprint(r)

## Exercices

Nous allons utiliser la base de données `test`, dans laquelle se trouve la collection `restaurants`. Celle-ci est composée de plus de 25000 retaurants new-yorkais, avec des évaluations de ceux-ci.

### 1. Créer une connexion vers cette collection

On peut complétement simplfier l'écriture des requêtes à suivre en faisant la connexion directement avec la collection `restaurants`.

In [4]:
db = con.test.restaurants

### 2. Déterminer le nombre exact de restaurants.

Un simple dénombrement est à faire ici.

In [5]:
db.count()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


25359

### 3. Lister les différents quartiers représentés (*borough*) ainsi que les différents types de  cuisine (*cuisine*).

On utilise la fonction `distinct()` ici pour lister les différentes valeurs de `borough`. On remarque que pour certains restaurants le quartier n'est pas connu.

In [6]:
db.distinct("borough")

['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island', 'Missing']

Idem pour lister les (nombreux) différents types de cuisine. On stocke le résultat dans une variable pour dénombrer les styles (`85`). Et nous n'affichons que les 10 premiers pour exemple.

In [7]:
type_cuisine = db.distinct("cuisine")
len(type_cuisine)

85

In [8]:
type_cuisine[:10]

['Bakery',
 'Hamburgers',
 'Irish',
 'American',
 'Jewish/Kosher',
 'Delicatessen',
 'Ice Cream, Gelato, Yogurt, Ices',
 'Chinese',
 'Chicken',
 'Turkish']

### 4. Lister les noms des restaurants, ainsi que leur quartier, ne proposant que des soupes (*cuisine* égale à *Soups*).

C'est la fonction `find()` qui nous permet de récupérer ces informations (4 restaurants).

In [9]:
res = db.find({ "cuisine": "Soups" }, { "_id": 0, "name": 1, "borough": 1})
list(res)

[{'borough': 'Manhattan', 'name': 'Wichcraft Express'},
 {'borough': 'Staten Island', 'name': 'Original Soupman Of Staten Island'},
 {'borough': 'Manhattan', 'name': 'The Original Soupman'},
 {'borough': 'Manhattan', 'name': 'Peasant Stock'}]

### 5. Donner le nombre de restaurants ayant eu au moins une fois le *grade* (dans *grades*) `Z`.

Pour cela, nous pouvons déjà nous retreindre aux restaurants ayant eu le *grade* `Z`, puis nous comptons la taille du résultat avec `count()`.

In [10]:
db.find({ "grades.grade": "Z"}).count()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


1337

### 6. Calculer le nombre de restaurants pour chaque quartier

Ici, un aggrégat basique est à faire, avec `aggregate()`. La plupart sont dans *Manhattan*, mais on remarque aussi qu'on a 51 restaurants sans quartier connu.

In [11]:
res = db.aggregate([ { "$group": { "_id": "$borough", "nb": { "$sum": 1 } } } ])
pandas.DataFrame(list(res)).rename(columns = {"_id": "Quartier", "nb": "Nb restaurants"})

,Quartier,Nb restaurants
0,Missing,51
1,Staten Island,969
2,Brooklyn,6086
3,Bronx,2338
4,Queens,5656
5,Manhattan,10259


### 7. Donner les cinq types de cuisine les plus présentes dans New-York (avec le plus de restaurants donc)

En plus d'un calcul d'aggrégat avec `$group`, on tri le résultat par ordre décroissant sur le nombre et on se limite aux 5 premières valeurs avec `$limit`.

In [12]:
res = db.aggregate([ 
    { "$group": { "_id": "$cuisine" , "nb" : { "$sum" : 1 } } },
    { "$sort" : { "nb": -1 } },
    { "$limit" : 5 }
])
pandas.DataFrame(list(res)).rename(columns = {"_id": "Cuisine", "nb": "Nb restaurants"})

,Cuisine,Nb restaurants
0,American,6183
1,Chinese,2418
2,Café/Coffee/Tea,1214
3,Pizza,1163
4,Italian,1069


### 8. Pour chaque *grade* possible, donner le *score* moyen, minimum et maximum.

Il faut d'abord dégrouper les évaluations pour chaque restaurant grâce à `$unwind`. Ensuite, c'est un calcul assez simple à réaliser.

In [13]:
res = db.aggregate([
    { "$unwind" : "$grades"},
    { "$group" : { 
        "_id": "$grades.grade", 
        "nb" : { "$sum" : 1 },
        "moy" : { "$avg" : "$grades.score" },
        "min" : { "$min" : "$grades.score" },
        "max" : { "$max" : "$grades.score" }
    } }
])
pandas.DataFrame(list(res), columns = ["_id", "nb", "moy", "min", "max"]).rename(columns = {
    "_id": "Grade", "nb": "Nb restaurants",
    "moy": "Score moyen", "min": "Score  minimum", "max": "Score maximum"
}).round(2)

,Grade,Nb restaurants,Score moyen,Score minimum,Score maximum
0,P,1197,5.51,0,43
1,A,74656,9.02,0,27
2,Not Yet Graded,525,15.85,0,75
3,B,12603,20.00,-1,57
4,Z,1337,23.70,-1,71
5,C,3145,30.70,-1,131


### 9. Créer un `DataFrame` contenant les restaurants en ligne et en colonne le score moyen obtenu pour chaque *grade* possible. Si un restaurant n'a pas eu d'évaluation d'un *grade*, nous aurons une valeur absente.

Si nous regardons le nombre de restaurants par nom de ceux-ci, nous remarquons qu'il y a des restaurants avec le même nom (des chaînes comme *Starbucks* ou *Subway* par exemple).

In [14]:
res = db.aggregate([
    { "$group": { "_id": "$name", "nb": { "$sum": 1 } } },
    { "$sort": { "nb": -1 } },
    { "$limit": 10 }
])
pandas.DataFrame(list(res))

,_id,nb
0,Subway,421
1,Starbucks Coffee,223
2,Mcdonald'S,208
3,Dunkin' Donuts,206
4,Dunkin Donuts,200
5,,151
6,Kennedy Fried Chicken,88
7,Crown Fried Chicken,81
8,Burger King,80
9,Domino'S Pizza,67


Nous allons donc faire un regroupement par identifiant de restaurant (`restaurant_id`) et par `grade`, pour calculer le score moyen. Nous voyons ici les premiers résultats de l'aggrégation.

In [15]:
res = db.aggregate([
    { "$unwind": "$grades" },
    { "$group": { 
        "_id": { "id": "$restaurant_id", "grade": "$grades.grade" },
        "m": { "$avg": "$grades.score" }
    }}
])

Ensuite, on créé un `DataFrame` à partir du résultat, en récupérant les sous-variables *à la main*. Ce qui nous donne à la fin le `DataFrame` suivant.

In [16]:
df = pandas.DataFrame(list(res))
df["ID"] = [id["id"] for id in df._id]
df["Grade"] = [id["grade"] for id in df._id]
df.head().round(2)

,_id,m,ID,Grade
0,"{'id': '50018661', 'grade': 'Not Yet Graded'}",2.0,50018661,Not Yet Graded
1,"{'id': '50018622', 'grade': 'Not Yet Graded'}",19.0,50018622,Not Yet Graded
2,"{'id': '50018608', 'grade': 'A'}",12.0,50018608,A
3,"{'id': '50018587', 'grade': 'Not Yet Graded'}",2.0,50018587,Not Yet Graded
4,"{'id': '50018565', 'grade': 'Not Yet Graded'}",0.0,50018565,Not Yet Graded


Il faut maintenant transformer ce `DataFrame` en utilisant la fonction `pivot_table()` du module `pandas`. Cette fonction prend quatre paramètres :

- `df` : le `DataFrame` sur lequel faire le pivot
- `index` : identifiant de chaque ligne (ici, l'identifiant des restaurants)
- `columns` : noms des colonnes (ici, chaque grade) 
- `value` : valeurs des cellules (ici, score moyen pour un restaurant pour un grade)

In [17]:
dfbis = pandas.pivot_table(df, index = 'ID', columns = 'Grade', values = 'm')
dfbis.head(10).round(1)

Grade,A,B,C,Not Yet Graded,P,Z
ID,,,,,,
30075445,6.8,14.0,NaN,NaN,NaN,NaN
30112340,10.7,23.0,NaN,NaN,NaN,NaN
30191841,9.2,NaN,NaN,NaN,NaN,NaN
40356018,9.0,NaN,NaN,NaN,NaN,NaN
40356068,13.0,25.0,NaN,NaN,NaN,20.0
40356151,10.0,NaN,NaN,NaN,NaN,38.0
40356442,10.5,NaN,NaN,NaN,NaN,NaN
40356483,10.0,NaN,NaN,NaN,NaN,NaN
40356649,9.6,NaN,NaN,NaN,NaN,NaN


Au final, nous regardons les informations de notre `DataFrame` ainsi créé. Nous remarquons que la plupart ont eu un grade `A`, et plusieurs n'ont pas encore été évalué.

In [18]:
dfbis.shape

(24608, 6)

In [19]:
dfbis.count()

Grade
A                 23440
B                  8280
C                  2708
Not Yet Graded      512
P                  1154
Z                  1337
dtype: int64

Nous récupérons ici les informations des restaurants (nom - `name` - et quartier - `borough`) pour les ajouter à notre table précédente. Ici, nous récupérons juste les informations que l'on souhaite.

In [20]:
infos = db.find({}, { "_id": 0, "restaurant_id": 1, "name": 1, "borough": 1 })
infos_df = pandas.DataFrame(list(infos)).rename(columns = { "restaurant_id": "ID", "name": "Nom", "borough": "Quartier" })
infos_df.head()

,Quartier,Nom,ID
0,Bronx,Morris Park Bake Shop,30075445
1,Brooklyn,Wendy'S,30112340
2,Manhattan,Dj Reynolds Pub And Restaurant,30191841
3,Brooklyn,Riviera Caterer,40356018
4,Queens,Tov Kosher Kitchen,40356068


Pour réaliser une jointure, nous avons la fonction `join()` qui réalise une jointure sur les `index` des lignes par défaut. La fonction `set_index()` permet de définir cet `index` de lignes pour les informations des restaurants, avant de faire la jointure.

In [21]:
final = dfbis.join(infos_df.set_index("ID"))
final.head()

,A,B,C,Not Yet Graded,P,Z,Quartier,Nom
ID,,,,,,,,
30075445,6.750000,14.0,NaN,NaN,NaN,NaN,Bronx,Morris Park Bake Shop
30112340,10.666667,23.0,NaN,NaN,NaN,NaN,Brooklyn,Wendy'S
30191841,9.250000,NaN,NaN,NaN,NaN,NaN,Manhattan,Dj Reynolds Pub And Restaurant
40356018,9.000000,NaN,NaN,NaN,NaN,NaN,Brooklyn,Riviera Caterer
40356068,13.000000,25.0,NaN,NaN,NaN,20.0,Queens,Tov Kosher Kitchen


Avec un `DataFrame` ainsi créé, il est maintenant possible, par exemple, de calculer les scores moyens de chaque grade, pour chaque quartier (ici, dans `python`).

In [22]:
res = final.groupby("Quartier").mean()
res.round(2)

,A,B,C,Not Yet Graded,P,Z
Quartier,,,,,,
Bronx,8.84,19.79,28.90,14.05,5.67,22.38
Brooklyn,8.94,19.87,28.73,16.41,5.35,23.45
Manhattan,8.94,19.85,31.18,17.84,5.68,24.49
Missing,8.41,22.00,43.50,NaN,NaN,NaN
Queens,9.12,19.88,31.12,14.22,5.28,23.51
Staten Island,9.33,19.37,32.82,12.85,6.68,19.77
